In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Missing").getOrCreate()

In [3]:
spark

In [5]:
training = spark.read.csv("./data/test1.csv",header=True,inferSchema=True)

In [6]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [7]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [9]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [11]:
from pyspark.ml.feature import VectorAssembler

`[Age, Experience]` ---> new feature ---> independet feature

In [14]:
featureassembler = VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [15]:
output=featureassembler.transform(training)

In [16]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [17]:
finalized_data=output.select("Independent Features","Salary")

In [18]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [19]:

from pyspark.ml.regression import LinearRegression


In [20]:
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])


In [21]:
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [23]:
### Coefficients
regressor.coefficients

DenseVector([-90.5483, 1608.7819])

In [27]:
### Intercepts
regressor.intercept

16079.136690647425

In [28]:
### Prediction
pred_results = regressor.evaluate(test_data)

In [30]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17214.09079632846|
+--------------------+------+-----------------+



In [31]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(785.909203671541, 617653.2764156357)